## Deployment - Batch prediction examples

In [1]:
pd.options.display.max_colwidth=10000

NameError: name 'pd' is not defined

In [4]:
import pandas as pd
pd.options.display
import datarobot as dr
from datetime import datetime
import time
import os
import io
import requests
import json
from pprint import pprint as pp

USERNAME = os.environ['DATAROBOT_USERNAME']
API_KEY = os.environ['DATAROBOT_API_TOKEN']
DATAROBOT_KEY = os.environ['DATAROBOT_KEY']
HOSTNAME = 'https://app.datarobot.com/' # The host to use for the REST API
ENDPOINT = HOSTNAME + 'api/v2/' # The host to use for the REST API

dr.Client(token=API_KEY, endpoint=ENDPOINT)  

In [6]:
#
# Get the dataset for predictions
#
pred_file = './data/DR_Demo_10K_Lending_Club_Loans_pred.csv'
df_pred_full = pd.read_csv(pred_file)   # there may be a limit to the number of rows 

# Encode all str columns as utf8
str_cols = df_pred_full.select_dtypes(include=['object']).columns
str_cols
for column in str_cols:
#     df[column] = df_peru[column].str.encode('utf-8')
#     print(column +' - ' )
    print(column)
    df_pred_full[column] = df_pred_full[column].str.encode('utf-8')

    
df_pred_full['desc'] = df_pred_full.desc.str.encode('utf-8')
                                   # that can be passed in to BatchPredictionJob.score
df_pred_subset = df_pred[:180]  #pd.read_csv(pred_file)[:13]

df_pred_fail = df_pred[179:180]

df_pred = df_pred_subset
df_pred = df_pred_full
# df_pred = df_pred_fail

print('df_pred.shape:', df_pred.shape)
df_pred

term
int_rate
grade
sub_grade
emp_title
emp_length
home_ownership
verification_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
earliest_cr_line
initial_list_status


TypeError: Cannot use .str.encode with values of inferred dtype 'bytes'.

In [ ]:
df_pred.dtypes
str_cols = df_pred.select_dtypes(include=['object']).columns
str_cols

In [75]:
str_cols = df_pred.select_dtypes(include=['object']).columns
str_cols
for column in str_cols:
#     df[column] = df_peru[column].str.encode('utf-8')
    print(column +' - ' )

term - 
int_rate - 
grade - 
sub_grade - 
emp_title - 
emp_length - 
home_ownership - 
verification_status - 
pymnt_plan - 
url - 
desc - 
purpose - 
title - 
zip_code - 
addr_state - 
earliest_cr_line - 
initial_list_status - 


In [63]:
df_pred_fail.to_dict()

{'loan_amnt': {179: 20000},
 'funded_amnt': {179: 20000},
 'term': {179: ' 36 months'},
 'int_rate': {179: '14.22%'},
 'installment': {179: 685.69},
 'grade': {179: 'C'},
 'sub_grade': {179: 'C5'},
 'emp_title': {179: 'Infinite Solutions'},
 'emp_length': {179: '4 years'},
 'home_ownership': {179: 'MORTGAGE'},
 'annual_inc': {179: 62000.0},
 'verification_status': {179: 'not verified'},
 'pymnt_plan': {179: 'n'},
 'url': {179: 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=490965'},
 'desc': {179: b'  Borrower added on 03/11/10 > We will be using the money to pay for our wedding expenses. We will be getting married at the coonamessett Inn in Falmouth, MA. The venue is approximately 16,000 including food of course. The flowers are 2,000, the dress is 1,000, invitations are another 250, photographer is 2500, the Dj was 1,500.the tux was 150, and the jp is 500 that\xc2\x92\xc2\xa2\xc3\x8e\xc2\x97\xc3\x8e\xc3\x88s the break down its aprox a 30,000 wedding but you only do it 

In [51]:
#
# Get the most recent lending club deployment
#

# Get deployments by search string
deployments_lc = dr.Deployment.list(search='is_bad Predictions')
deployments_lc

deployment = deployments_lc[-1]
print(deployment.id)

5f6e3af7684b9709659b5dce


In [52]:
pp(deployment.model)
print()

project_id = deployment.model.get('project_id')
print('project_id:', project_id)

model_id = deployment.model.get('id')
print('model_id:', model_id)

{'build_environment_type': 'DataRobot',
 'deployed_at': '2020-09-25T18:46:15.629000Z',
 'id': '5f64f7c8758c297b8c29e3ab',
 'project_id': '5f64f3a656f28d076e6723cb',
 'project_name': '10K Lending Club Loans.csv',
 'target_name': 'is_bad',
 'target_type': 'Binary',
 'type': 'Light Gradient Boosted Trees Classifier with Early Stopping',
 'unstructured_model_kind': False,
 'unsupervised_mode': False}

project_id: 5f64f3a656f28d076e6723cb
model_id: 5f64f7c8758c297b8c29e3ab


In [5]:
#
# FYI, the BATCH_PREDICTIONS_URL = 'https://app.datarobot.com/api/v2'
#

In [6]:
#
# Batch scoring synchronously (Program flow waits for call to return).  
# - If output_settings includes a 'file:' element, the call waits for completion before returning program flow.
#
# With prediction explanations if uncommented as shown. 
#
t0 = time.time()
dr.BatchPredictionJob.score(
    deployment.id,
    intake_settings={
        'type': 'localFile',
        'file': './data/DR_Demo_10K_Lending_Club_Loans_pred.csv' # Path or Pandas or file-like object.  
                                                                 # Such as: 'file': df_pred_data
    },
    output_settings={
        'type': 'localFile',
        'path': 'outputfile.csv'
    },
    download_timeout=6000,
    # If explanations are required, uncomment the line below
    # max_explanations=3,
    # Uncomment this for Prediction Warnings, if enabled for your deployment.
    # prediction_warning_enabled=True
)
print('- Execution time: %.3f min' % ((time.time() - t0)/60))

df_results = pd.read_csv('./outputfile.csv')
df_results

KeyboardInterrupt: 

In [8]:
#
# Batch scoring asynchronously (Program flow waits for call to return).  
#

# Helper function to check the status of the prediction job
def check_job_status(job, sleep=10):
    """
    After a batch prediction request is posted asynchronously, the call retuns a jobid
    and program flow returns.  This helper function checks the request processing status and
    prints current state
    
    Parameters:
    - job id
    - sleep: the time to wait between status checks
    
    Returns:
    - nothing
    """

    out_str = ''
    
    def output(txt, add_flush=True):       
        if add_flush:
            print(txt, end='\r', flush=True)
        else:
            print(txt)

    job_status = job.get_status()

    while True: 
        job_status = job.get_status()
        elapsed_time = job_status['elapsed_time_sec']
        status = job_status.get('status')
        status_details = job_status['status_details']
        
        out_str = "Wait time {:.3f} minutes - Status {}. "  \
                  "Queue posiition: {} - {}".format(elapsed_time/60, 
                                                                                           status,
                                                                                           job._safe_data.get('queue_position'),
                                                                                           status_details)

        if job.get_status().get('status') == 'INITIALIZING':
            str_split = status_details.split()
            
            try:
                job_index = str_split.index("job(s)")
                num_jobs_ahead = str_split[job_index-1]
                status_details = str(num_jobs_ahead+' job(s) ahead')
            except Exception as e:
                status_details = '0 job(s) ahead'

            output(out_str)
        else:
            print()
            output(out_str, add_flush=False)

            if job.get_status().get('status') == 'COMPLETED' or \
               job.get_status().get('status') == 'ABORTED':
                    break
            
        time.sleep(sleep)

In [65]:
#
# Batch scoring asynchronously (Program flow returns a job id, poll the job for completion).  
#
job = dr.BatchPredictionJob.score(
    deployment.id,
#     prediction_instance={
#         'hostName': host.url,
#     },
    intake_settings={
        'type': 'localFile',
#         'file': './data/DR_Demo_10K_Lending_Club_Loans_pred.csv',  # pred file as csv
        'file': df_pred,  # pred file as dataframe
    },
    output_settings={
        'type': 'localFile',
    },
)

# Print the job object
print('job attributes:')
pp(job.__dict__)

job attributes:
{'_completed_resource_url': None,
 '_safe_data': {'created': '2022-04-20T15:34:57.179000Z',
                'created_by': {'full_name': 'Matthew Cohen',
                               'user_id': '5a8a6402b11ba422e62b7c7a',
                               'username': 'matthew.cohen@datarobot.com'},
                'elapsed_time_sec': 1,
                'failed_rows': 0,
                'id': '626028211b10eba4835a54d4',
                'job_spec': {'abort_on_error': True,
                             'chunk_size': 'auto',
                             'csv_settings': {'delimiter': ',',
                                              'encoding': 'utf-8',
                                              'quotechar': '"'},
                             'deployment_id': '5f6e3af7684b9709659b5dce',
                             'disable_row_level_error_handling': False,
                             'include_prediction_status': False,
                             'include_probabilities'

In [ ]:
check_job_status(job, sleep=3)
print()

print('Results:')
result_bytes = job.get_result_when_complete()
result_str = result_bytes.decode("utf-8") 
result_io = io.StringIO(result_str)
# print(result_str)
df = pd.read_csv(result_io, sep=",")
df